In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<img src="https://media.giphy.com/media/HwvXsuDiQLaiQ/giphy.gif">

![](http://www.google.com/url?sa=i&url=http%3A%2F%2Fjoyreactor.com%2Ftag%2Fgif%2Fbest%2F450&psig=AOvVaw0viirU0t1DG_QCEVFfZk4W&ust=1616887773681000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCIjVmL2Oz-8CFQAAAAAdAAAAABAW)

![](http://https://www.google.com/url?sa=i&url=http%3A%2F%2Fjoyreactor.com%2Ftag%2Fgif%2Fbest%2F450&psig=AOvVaw0viirU0t1DG_QCEVFfZk4W&ust=1616887773681000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCIjVmL2Oz-8CFQAAAAAdAAAAABAW)

In [ ]:
emissions = pd.read_csv('/kaggle/input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv')
emissions.head()

# DATASET DESCRIPTION
This dataset captures the details of how CO2 emissions by a vehicle can vary with the different features. The columns of this dataset are 
1. Model
2. Make	
3. Engine Size(L)	
4. Cylinders
5. Fuel Type
6. Fuel Consumption City (L/100 km)
7. Fuel Consumption Hwy (L/100 km)	
8. Fuel Consumption Comb (mpg and L/100 km)
9. Vehicle class
10. Transmission

# Columns

In [ ]:
emissions.columns

# Rename columns as there are many gaps, which creates un-necessary errors

In [ ]:
renamed_col = {
    'Vehicle Class': 'vehicle_class',
    'Engine Size(L)': 'engine_size',
    'Fuel Type': 'fuel_type',
    'Fuel Consumption City (L/100 km)': 'fuel_cons_city',
    'Fuel Consumption Hwy (L/100 km)': 'fuel_cons_hwy',
    'Fuel Consumption Comb (L/100 km)': 'fuel_cons_comb',
    'Fuel Consumption Comb (mpg)': 'mpgfuel_cons_comb',
    'CO2 Emissions(g/km)': 'co2'
}
emissions.rename(renamed_col, axis='columns', inplace=True)

In [ ]:
emissions.keys()

# Data-analysis

In [ ]:
emissions.info()

**Lookss like there are no missing values and the data-types are appropriate** :D 
The dataset contains 7385 observations with 12 columns, with data-types float,int and object.

In [ ]:
emissions.describe().transpose()

# EDA
*We will explore the features by statistical distributions to find any outliers or skewness.From the EDA, we get to know which features will be relevant in the predictions of the target variable and thus we can stop considering less relevant features*                                                                
**EDA let's you get to know about your dataset. So, try as much explorations which will result in better assumptions in creating the model**

# Import lib for visualisations


In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import express as px, graph_objects as go
from plotly.subplots import make_subplots

# Why not start with the 1st column 'Make'

In [ ]:
emissions.Make.unique()

In [ ]:
f, ax = plt.subplots(figsize=(25,7))
sns.set_theme(style="darkgrid")

x = emissions.Make.value_counts().sort_values()

ax = sns.barplot(data=emissions,x='Make',y='co2')
plt.title('Vehicles and their Co2 emissions')
plt.xticks(rotation=35)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(25,7))
sns.set_theme(style="darkgrid")

x = emissions.Make.value_counts().sort_values()

ax = sns.barplot(data=emissions,x='Make',y='co2', hue='Cylinders')
plt.title('Number of cylinders in respective vehicles')
plt.xticks(rotation=35)
plt.show()

# Interesting, highest co2 emitter Bugetti seems to have maximum num of cylinders - 16.. We will see how these variables correlate

**Bugatti has highest co2 emissions more than 500 while SMART seems to have lwest of 150**

# Copy our dataframe to a new variable

In [ ]:
df = emissions.copy()

renamed_col = {
    'Vehicle Class': 'vehicle_class',
    'Engine Size(L)': 'engine_size',
    'Fuel Type': 'fuel_type',
    'Fuel Consumption City (L/100 km)': 'fuel_cons_city',
    'Fuel Consumption Hwy (L/100 km)': 'fuel_cons_hwy',
    'Fuel Consumption Comb (L/100 km)': 'fuel_cons_comb',
    'Fuel Consumption Comb (mpg)': 'mpgfuel_cons_comb',
    'CO2 Emissions(g/km)': 'co2'
}
df.rename(renamed_col, axis='columns', inplace=True)

# Retrieve the maximum co2 emissions and their make

In [ ]:
df[df.co2 == df.co2.max()]

In [ ]:
df[df.Make == 'SMART']

In [ ]:
fig = px.histogram(df, x="Cylinders")
fig.update_layout(title_text='Cylinders in Vehicles',xaxis_title="Cylinders ",yaxis_title="Number_of_vehicles ",title_x=0.5)
fig.show()

In [ ]:
df.Cylinders.value_counts(normalize=True)

In [ ]:
df.Cylinders.value_counts()

# 43.5% of the vehicles have 4 cylinders followed by 6 cylinders and 8. Only 3  vehicles have highest number of cylinders - 16

In [ ]:
lamborghini = df[df.Make == 'LAMBORGHINI']
lamborghini_models = lamborghini.groupby('Model').count().reset_index().sort_values('Make', ascending=False)
lamborghini_models
f, ax = plt.subplots(figsize=(25,7))
sns.set_theme(style="whitegrid")
ax = sns.barplot(data=lamborghini_models, y='Make',x='Model')
plt.title('Number of models in Lamborghini')
plt.xticks(rotation=90)
plt.show()

# view the median, IQR for various type of vehicles

In [ ]:
plt.figure(figsize = (15,8))
sns.boxplot(x="Make", y="co2", data=df)
plt.xticks(rotation = 90)
plt.show()

**we could see that all models of Rolls-royce has emissions more than 350. Some models(outliers) in Mercedes-Benz and FORD are competing with the maximum emissions**

In [ ]:
df.head()

In [ ]:
sns.jointplot(
    data=df,
    x="fuel_cons_city", y="co2", hue="fuel_type",
    kind="kde"
)
plt.show()

# Plotting Transmission Distribution

In [ ]:
transmission_distr =df['Transmission'].value_counts().reset_index()
fig = px.pie(transmission_distr, values='Transmission', names='index')
fig.update_layout(title="Transmission Distribution")
fig.show()

In [ ]:
plt.figure(figsize = (15,8))
sns.countplot(y="fuel_type", data=df, hue='Cylinders')
plt.title('Distribution of Fuel-Type')
plt.show()

In [ ]:
fig = go.Figure(data=go.Violin(y=df["fuel_cons_city"],
                               box_visible=True,
                               line_color='white',
                               meanline_visible=True,
                               fillcolor='blue', opacity=0.4,
                               x0='Fuel Consumption City'))

fig.show()

In [ ]:
df.head(3)

# Fuel consumption in City and Highway

In [ ]:
city=df.fuel_cons_city
highway = df.fuel_cons_hwy
city_hwy_df = df[['fuel_cons_city', 'fuel_cons_hwy', 'fuel_type','co2']]
city_hwy_df.head()

In [ ]:
fig = plt.figure(figsize = (10,20))
ax = fig.add_subplot(2,1,1)
ax = plt.hist(city, color="orange", label="Fuel Consumption in city", density=True, alpha=0.5)
ax = plt.hist(highway, color="blue", label="Fuel Consumption in Highway", density=True, alpha=0.5)
ax = plt.legend()
plt.show()

# **Obviously as one would expect, Fuel consumption in Highways are higher**

In [ ]:
sns.kdeplot(
   data=city_hwy_df, x="fuel_cons_hwy", hue="fuel_type",
   fill=True, common_norm=False, palette="crest",
   alpha=.5, linewidth=0,
)

# Determine the most used fuel_type which has key role in the co2 emisssions

In [ ]:
sns.histplot(city_hwy_df, x="fuel_cons_hwy", hue="fuel_type", element="poly")
plt.title('Fuel types used in Highways')
plt.show()

In [ ]:
sns.histplot(city_hwy_df, x="fuel_cons_city", hue="fuel_type", element="step")
plt.title('Fuel types used in City')
plt.show()

# Fuel types Z and X are used dominantly both in highways and city which are         "Premium gasoline" and "regular gasoline" 

In [ ]:
df.head()

# Distributions of Combined Fuel consumption in (L/100km)  and mpg

In [ ]:
sns.histplot(data=df, x="mpgfuel_cons_comb", bins=30)
plt.title('Combined Fuel consumption in (L/100km)')
plt.show()

In [ ]:
sns.histplot(data=df, x="fuel_cons_comb", bins=30)
plt.title('Combined Fuel consumption in mpg')
plt.show()

# As we see from the histograms, the distribution is near symmetrical and less skewed. SO,we need not apply Log transformations..

In [ ]:
sns.histplot(df, x="engine_size", y="co2")

# Correlations in the data

In [ ]:
cor = df.corr()
sns.heatmap(cor)

In [ ]:
df.corr()

# Simple Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, 
                                   PolynomialFeatures)

In [ ]:
slr = LinearRegression()

In [ ]:
y_col = "co2"

X = df.drop(y_col, axis=1)
y = df[y_col]

In [ ]:
X = df[['engine_size']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=72018) #split into train and test parts

In [ ]:
s = StandardScaler()
X_train_s = s.fit_transform(X_train)

In [ ]:
slr.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = slr.predict(X_test_s)

In [ ]:
#Accuracy and MSE
print(f'R2 score is {r2_score(y_pred,y_test)}')
print("Residual sum of squares (MSE): %.2f" % np.mean((y_pred - y_test) ** 2))

# 65% accuracy

# Multiple Linear Regression

In [ ]:
mlr = LinearRegression()
X = df[['engine_size', 'Cylinders', 'fuel_cons_comb']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=72018) #split into train and test parts
X_train_sm = s.fit_transform(X_train)
mlr.fit(X_train_sm, y_train)
X_test_sm = s.transform(X_test)
y_pred_sm = mlr.predict(X_test_sm)

In [ ]:
#Accuracy and MSE
print(f'R2 score is {r2_score(y_pred_sm,y_test)}')
print("Residual sum of squares (MSE): %.2f" % np.mean((y_pred_sm - y_test) ** 2))

In [ ]:
print(f'Co-efficients are {mlr.coef_}')
print(f'Intercept is {mlr.intercept_}')

# We trained upto 86% accurately predicting model

# Cross-Validation


In [ ]:
from sklearn.model_selection import cross_val_score
Rcross = cross_val_score(slr, X_train_s, y_train, cv=4)
print("The mean of the folds are", Rcross.mean(), "and the standard deviation is" , Rcross.std())

**cross validation for multiple linear regr**

In [ ]:
from sklearn.model_selection import cross_val_score
Rcross = cross_val_score(mlr, X_train_sm, y_train, cv=4)
print("The mean of the folds are", Rcross.mean(), "and the standard deviation is" , Rcross.std())

# Introduce Polynomial features

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
pf = PolynomialFeatures(degree=2, include_bias=False,)
X_pf = pf.fit_transform(X)
X_pf_ss = s.fit_transform(X_pf)

**The polynomial features will make model complex. It is better to use regularization techniques like Lasso, Ridge to see at what point the model overfits and the accuracy starts declining. You would want to stop training your model there**
REferences: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [ ]:
lasso = Lasso()
lasso.fit(X_pf_ss, y)
lasso.coef_ 

In [ ]:
r2_score(y,lasso.predict(X_pf_ss))

# Different alpha values

In [ ]:
las01 = Lasso(alpha = 0.1)
las01.fit(X_pf_ss, y)
print('sum of coefficients:', abs(las01.coef_).sum() )
print('number of coefficients not equal to 0:', (las01.coef_!=0).sum())

In [ ]:
las1 = Lasso(alpha = 1)
las1.fit(X_pf_ss, y)
print('sum of coefficients:',abs(las1.coef_).sum() )
print('number of coefficients not equal to 0:',(las1.coef_!=0).sum())

**conclusion- choose alpha = 1**

# Ridge

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
r = Ridge(alpha = 0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

# Calculating r2 score
r.coef_

*There are different model evaluation metrics
https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b, we will calculate the metrics given in this blog-post*
1. Mean absolute error - It is the mean of the absolute value of the errors.
2. Mean Squared Error - This is the popular metric and it is the mean of the squared error.
3. RMSE -  This calculates the accuracy of the model.